<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/ai_translation_tuning_sft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Supervised Tuning**

*Task: Automatically tune and evaluate translation models for multiple long-tail languages*


https://cloud.google.com/vertex-ai/generative-ai/docs/models/translation-supervised-tuning

In [ ]:
# Update project ID, GCP bucket name and name of TMX file manually:

PROJECT_ID = "YOUR-PROJECT-ID"               # <--- UPDATE THIS
LOCATION = "us-central1"
BUCKET_NAME = "translations-eval" # <--- UPDATE THIS
BUCKET_URI = f"gs://{BUCKET_NAME}"
TMX_GCS_PATH = "samples.tmx"    # <--- UPLOAD THIS
LOCAL_TMX_FILE = "samples.tmx"

###### *Setup and Dependencies*

In [ ]:
%pip install --upgrade google-cloud-aiplatform google-cloud-storage -q
%pip install matplotlib seaborn langdetect -q
%pip install --upgrade --user --quiet google-cloud-aiplatform[evaluation]

!pip install google-cloud-translate==2.0.1 -q
!pip install --upgrade google-cloud-translate -q

import xml.etree.ElementTree as ET
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from google.cloud import aiplatform, storage
from google.cloud import translate_v3 as translate
import vertexai
from vertexai.tuning import sft
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import re
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# For language detection
from langdetect import detect, detect_langs

In [ ]:
# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Utility Functions
def download_from_gcs(bucket_name, source_blob_name, destination_file_name):
    """Downloads a file from GCS and returns the local path."""
    print(f"--- Downloading {source_blob_name} ---")
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Successfully downloaded to {destination_file_name}")
    return destination_file_name

def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to GCS and returns the GCS URI."""
    print(f"--- Uploading {source_file_name} ---")
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    gcs_uri = f"gs://{bucket_name}/{destination_blob_name}"
    print(f"Successfully uploaded to {gcs_uri}")
    return gcs_uri

###### *Automatic **Preprocessing** of all Languages*

In [ ]:
# @title STEP 2: AUTOMATIC TXM PREPROCESSING FOR ALL LANGUAGES
# ============================================================================

def create_tuning_data_from_samples(samples, lang_code):
    """Convert samples to the required 'contents' format for tuning."""
    tuning_records = []
    for sample in samples:
        json_record = {
            "contents": [
                {"role": "user", "parts": [{"text": f"English: {sample['source']} {lang_code}: "}]},
                {"role": "model", "parts": [{"text": sample['target']}]}
            ]
        }
        tuning_records.append(json_record)
    return tuning_records

def preprocess_and_upload_language(lang_code, samples):
    """Preprocess and upload data for a single language."""
    print(f"\n Automatic Processing {lang_code}...")

    # Remove duplicates and short samples
    unique_samples = []
    seen_pairs = set()
    for sample in samples:
        pair = (sample['source'], sample['target'])
        if pair not in seen_pairs and sample['source_words'] >= 3 and sample['target_words'] >= 3:
            seen_pairs.add(pair)
            unique_samples.append(sample)

    print(f"   Filtered from {len(samples)} to {len(unique_samples)} unique quality samples")

    if len(unique_samples) < 10:
        print(f"   ⚠️ Skipping {lang_code} - insufficient samples after filtering")
        return None

    # Convert to tuning format
    tuning_records = create_tuning_data_from_samples(unique_samples, lang_code)

    # Split into train/eval
    train_data, eval_data = train_test_split(tuning_records, test_size=0.2, random_state=42)
    print(f"   Split: {len(train_data)} training, {len(eval_data)} evaluation")

    # Save and upload train data
    train_file_local = f"{lang_code.replace(':', '_')}_tuning_train_split.jsonl"
    with open(train_file_local, 'w', encoding='utf-8') as f:
        for item in train_data:
            f.write(json.dumps(item) + '\n')
    train_gcs_uri = upload_to_gcs(BUCKET_NAME, train_file_local, f"datasets/{train_file_local}")

    # Save and upload eval data
    eval_file_local = f"{lang_code.replace(':', '_')}_tuning_eval_split.jsonl"
    with open(eval_file_local, 'w', encoding='utf-8') as f:
        for item in eval_data:
            f.write(json.dumps(item) + '\n')
    eval_gcs_uri = upload_to_gcs(BUCKET_NAME, eval_file_local, f"datasets/{eval_file_local}")

    return {
        'lang_code': lang_code,
        'train_uri': train_gcs_uri,
        'eval_uri': eval_gcs_uri,
        'train_count': len(train_data),
        'eval_count': len(eval_data)
    }

print("\n" + "="*80)
print("STEP 2: PREPROCESSING AND UPLOADING ALL LANGUAGES")
print("="*80)

# Process all languages
preprocessing_results = []
if all_language_data:
    for lang_code, samples in all_language_data.items():
        result = preprocess_and_upload_language(lang_code, samples)
        if result:
            preprocessing_results.append(result)

    # Summary of preprocessing
    if preprocessing_results:
        print("\n Preprocessing Summary:")
        prep_df = pd.DataFrame(preprocessing_results)
        # Reorder columns for better readability
        prep_df = prep_df[['lang_code', 'train_count', 'eval_count']]
        prep_df.columns = ['Language', 'Training Samples', 'Evaluation Samples']
        prep_df['Total'] = prep_df['Training Samples'] + prep_df['Evaluation Samples']
        prep_df['Split'] = prep_df.apply(lambda x: f"{x['Training Samples']}/{x['Evaluation Samples']} (80/20)", axis=1)
        print(prep_df[['Language', 'Total', 'Split']].to_string(index=False))
    else:
        print("\n⚠️ No languages had sufficient data for preprocessing")
else:
    print("\n⚠️ No data available for preprocessing")


###### *Automatic **Supervised Tuning** of all Languages*

In [ ]:
# @title STEP 3: AUTOMATIC MODEL TUNING FOR ALL LANGUAGES
# ============================================================================

def launch_tuning_job(prep_result):
    """Launch a tuning job for a single language."""
    lang_code = prep_result['lang_code']

    # Create a safe model name
    safe_lang_code = lang_code.lower().replace('-', '_').replace(':', '_')
    model_display_name = f'tuned-translator-{safe_lang_code}'

    print(f"\n Launching tuning job for {lang_code}...")

    try:
        sft_tuning_job = sft.train(
            source_model="translation-llm-002",
            train_dataset=prep_result['train_uri'],
            validation_dataset=prep_result['eval_uri'],
            tuned_model_display_name=model_display_name,
        )

        print(f"   ✅ Job submitted for {lang_code}")
        print(f"   Job Name: {sft_tuning_job.resource_name}")

        return {
            'lang_code': lang_code,
            'job': sft_tuning_job,
            'job_name': sft_tuning_job.resource_name,
            'model_name': model_display_name,
            'status': 'submitted',
            'start_time': datetime.now()
        }
    except Exception as e:
        print(f"   ❌ Failed to submit job for {lang_code}: {str(e)}")
        return {
            'lang_code': lang_code,
            'job': None,
            'error': str(e),
            'status': 'failed'
        }

def monitor_tuning_jobs(tuning_jobs, check_interval=300):
    """Monitor all tuning jobs until completion."""
    print("\n Monitoring tuning jobs...")

    active_jobs = tuning_jobs.copy()
    completed_jobs = []

    while active_jobs:
        time.sleep(check_interval)

        for job_info in active_jobs[:]:
            if job_info['job'] is None:
                active_jobs.remove(job_info)
                completed_jobs.append(job_info)
                continue

            try:
                job_info['job'].refresh()

                if job_info['job'].has_ended:
                    job_info['status'] = 'completed'
                    job_info['end_time'] = datetime.now()
                    job_info['duration'] = job_info['end_time'] - job_info['start_time']

                    # Try to get endpoint
                    try:
                        job_info['endpoint'] = job_info['job'].tuned_model_endpoint_name
                    except:
                        job_info['endpoint'] = 'N/A'

                    print(f"\n✅ {job_info['lang_code']} completed!")
                    print(f"   Duration: {job_info['duration']}")
                    print(f"   Endpoint: {job_info['endpoint']}")

                    active_jobs.remove(job_info)
                    completed_jobs.append(job_info)

            except Exception as e:
                print(f"\n Error checking {job_info['lang_code']}: {str(e)}")

        if active_jobs:
            print(f"\n Still running: {', '.join([j['lang_code'] for j in active_jobs])}")

    return completed_jobs

print("\n" + "="*80)
print("STEP 3: MODEL TUNING FOR ALL LANGUAGES")
print("="*80)

if preprocessing_results:
    # Launch all tuning jobs
    print("\n Launching tuning jobs for all languages...")
    tuning_jobs = []

    for prep_result in preprocessing_results:
        job_info = launch_tuning_job(prep_result)
        tuning_jobs.append(job_info)
        time.sleep(2)  # Small delay between submissions

    # Save tuning job information for later use in evaluation
    tuning_jobs_file = "tuning_jobs_info.json"
    with open(tuning_jobs_file, 'w') as f:
        # Convert to serializable format
        jobs_data = []
        for job in tuning_jobs:
            job_data = {
                'lang_code': job['lang_code'],
                'job_name': job.get('job_name', ''),
                'model_name': job.get('model_name', ''),
                'status': job['status']
            }
            jobs_data.append(job_data)
        json.dump(jobs_data, f, indent=2)
    print(f"\n Saved tuning job information to {tuning_jobs_file}")

    # Print summary
    print("\n" + "="*80)
    print("TUNING JOBS SUMMARY")
    print("="*80)

    # Create tuning summary table
    tuning_summary = []
    for job in tuning_jobs:
        tuning_summary.append({
            'Language': job['lang_code'],
            'Status': '✅ ' + job['status'] if job['status'] == 'submitted' else '❌ ' + job['status'],
            'Model Name': job.get('model_name', 'N/A'),
            'Job ID': job.get('job_name', 'N/A').split('/')[-1] if job.get('job_name') else 'N/A',
            'Error': job.get('error', '')[:50] if job.get('error') else ''
        })

    tuning_df = pd.DataFrame(tuning_summary)
    print("\n" + tuning_df[['Language', 'Status', 'Model Name', 'Job ID']].to_string(index=False))

    # Print any errors
    errors = [job for job in tuning_jobs if job.get('error')]
    if errors:
        print("\n⚠️ Errors encountered:")
        for job in errors:
            print(f"   {job['lang_code']}: {job['error']}")

    print("\n Notes:")
    print("- Monitor progress in the Vertex AI console using the Job IDs above")
    print("- Training typically takes 2-4 hours per model")
    print("- You'll receive an email notification when each job completes")
    print("- Tuning job info saved for automatic evaluation later")

    # completed_jobs = monitor_tuning_jobs(tuning_jobs, check_interval=300)
else:
    print("\n⚠️ No preprocessing results available for tuning")

# Helper function to check job status later
def check_job_status(job_resource_name):
    """Check the status of a specific tuning job."""
    try:
        job = sft.SupervisedTuningJob(job_resource_name)
        print(f"Job: {job_resource_name}")
        print(f"Status: {'Completed' if job.has_ended else 'Running'}")
        if job.has_ended:
            print(f"Endpoint: {job.tuned_model_endpoint_name}")
    except Exception as e:
        print(f"Error: {str(e)}")

print("\n✅ Pipeline execution complete!")
print("\nNext steps:")
print("1. Monitor tuning jobs in the Vertex AI console")
print("2. Once complete, use the endpoints for inference")
print("3. Evaluate model performance on test data")

INFO:vertexai.tuning._tuning:Creating SupervisedTuningJob



STEP 3: MODEL TUNING FOR ALL LANGUAGES

 Launching tuning jobs for all languages...

 Launching tuning job for es-LA...


INFO:vertexai.tuning._tuning:SupervisedTuningJob created. Resource name: projects/892203813305/locations/us-central1/tuningJobs/3123832192300482560
INFO:vertexai.tuning._tuning:To use this SupervisedTuningJob in another session:
INFO:vertexai.tuning._tuning:tuning_job = sft.SupervisedTuningJob('projects/892203813305/locations/us-central1/tuningJobs/3123832192300482560')
INFO:vertexai.tuning._tuning:View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/3123832192300482560?project=892203813305


   ✅ Job submitted for es-LA
   Job Name: projects/892203813305/locations/us-central1/tuningJobs/3123832192300482560


INFO:vertexai.tuning._tuning:Creating SupervisedTuningJob


...

   ✅ Job submitted for fr-FR
   Job Name: projects/892203813305/locations/us-central1/tuningJobs/5828243768536465408


INFO:vertexai.tuning._tuning:Creating SupervisedTuningJob



 Launching tuning job for de-DE...


INFO:vertexai.tuning._tuning:SupervisedTuningJob created. Resource name: projects/892203813305/locations/us-central1/tuningJobs/9118123296330612736
INFO:vertexai.tuning._tuning:To use this SupervisedTuningJob in another session:
INFO:vertexai.tuning._tuning:tuning_job = sft.SupervisedTuningJob('projects/892203813305/locations/us-central1/tuningJobs/9118123296330612736')
INFO:vertexai.tuning._tuning:View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/9118123296330612736?project=892203813305


   ✅ Job submitted for de-DE
   Job Name: projects/892203813305/locations/us-central1/tuningJobs/9118123296330612736

 Saved tuning job information to tuning_jobs_info.json

TUNING JOBS SUMMARY

Language      Status             Model Name              Job ID
   es-LA ✅ submitted tuned-translator-es_la 3123832192300482560
   pt-BR ✅ submitted tuned-translator-pt_br 1970910687693635584
   fr-FR ✅ submitted tuned-translator-fr_fr 5828243768536465408
   de-DE ✅ submitted tuned-translator-de_de 9118123296330612736

 Notes:
- Monitor progress in the Vertex AI console using the Job IDs above
- Training typically takes 2-4 hours per model
- You'll receive an email notification when each job completes
- Tuning job info saved for automatic evaluation later

✅ Pipeline execution complete!

Next steps:
1. Monitor tuning jobs in the Vertex AI console
2. Once complete, use the endpoints for inference
3. Evaluate model performance on test data


###### *Automatic **Ground Truth Tables** for all languages*

In [ ]:
# @title Create ground-truth tables for all languages automatically

import pandas as pd
import json
from google.cloud import storage

# 1. Dynamically Get Languages from Preprocessing Step
# uses the 'preprocessing_results' created in Step 2, removing the need for a manual list.
if 'preprocessing_results' not in locals() or not preprocessing_results:
    raise NameError("'preprocessing_results' not found. Please run Step 2 first.")

print(f" Found {len(preprocessing_results)} languages from the preprocessing step. Starting table creation...")

# 2. Initialize a Dictionary to Store All DataFrames
ground_truth_dfs = {}
storage_client = storage.Client()


# 3. Loop Through Each Language
for lang_info in preprocessing_results:
    lang_code = lang_info['lang_code']
    gcs_uri = lang_info['eval_uri'] # Use the evaluation URI from the results
    print(f"\n--- Processing {lang_code} ---")

    # Download and Parse Data for the current language
    try:
        bucket_name, blob_name = gcs_uri.replace("gs://", "").split("/", 1)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        content = blob.download_as_text()
        eval_data = [json.loads(line) for line in content.strip().split('\n')]
        print(f" Loaded {len(eval_data)} records from GCS..")
    except Exception as e:
        print(f"❌ Failed to load data for {lang_code}. Error: {e}")
        continue # Skip to the next language if there's an error

    # Process records and create lists
    prompts = []
    references = []
    for item in eval_data:
        full_prompt_str = item['contents'][0]['parts'][0]['text']
        # Dynamically use the lang_code to clean the string
        english_text = full_prompt_str.replace("English: ", "").replace(f" {lang_code}: ", "").strip()
        prompts.append(english_text)

        reference_translation = item['contents'][1]['parts'][0]['text']
        references.append(reference_translation)

    # Create the DataFrame for the current language
    df = pd.DataFrame({
        "source_text": prompts,
        "reference": references,
    })

    # Store the new DataFrame in our main dictionary
    ground_truth_dfs[lang_code] = df
    print(f"✅ Ground truth DataFrame for {lang_code} created successfully.")


# 4. Display results
print("\n\n==============================================")
print("Automatic Processing Complete!")
print(f"Created {len(ground_truth_dfs)} tables, stored in the 'ground_truth_dfs' dictionary.")
print("==============================================")

# Display the top few rows of each created table
for lang_code, df in ground_truth_dfs.items():
    print(f"\n--- Table for: {lang_code} ---")
    display(df.head())

▶️ Found 4 languages from the preprocessing step. Starting table creation...

--- Processing es-LA ---
✅ Loaded 23 records from GCS.
✅ Ground truth DataFrame for es-LA created successfully.

--- Processing pt-BR ---
✅ Loaded 24 records from GCS.
✅ Ground truth DataFrame for pt-BR created successfully.

--- Processing fr-FR ---
✅ Loaded 22 records from GCS.
✅ Ground truth DataFrame for fr-FR created successfully.

--- Processing de-DE ---
✅ Loaded 24 records from GCS.
✅ Ground truth DataFrame for de-DE created successfully.


✅ Automatic Processing Complete!
Created 4 tables, stored in the 'ground_truth_dfs' dictionary.

--- Table for: es-LA ---


,source_text,reference
0,"About to say oh thank you, I was tell.","Pero, bueno, gracias."
1,They adapt to the fact that they it’s better f...,Se adaptan al hecho de que es mejor para la su...
2,"What is up, Daddy Gang?","¿Cómo están, seguidores de Daddy?"
3,"OK, so I was, I had just taken the LSAT. I wan...","Bueno, acababa de rendir el examen LSAT. Querí..."
4,"Of course, you know, it’s not me saying this, ...","Por supuesto, no soy yo quien dice esto. Hay u..."



--- Table for: pt-BR ---


,source_text,reference
0,Is it important?,Isso é importante?
1,But I didn't know we had such an impact.,Mas eu não sabia que nosso impacto era tão gra...
2,"You know, the fact is that if sleep wasn’t imp...","Sabe, o fato é que, se o sono não fosse import..."
3,"It is your founding father, Alex Cooper with.",Sou Alex Cooper e você está ouvindo
4,"OK, so I was, I had just taken the LSAT. I wan...","Tá, eu tinha acabado de fazer o LSAT. Eu queri..."



--- Table for: fr-FR ---


,source_text,reference
0,But that whole natural selection thing when it...,Tout ce processus de sélection naturelle pour ...
1,"I'm not kidding you. I was, and I think the en...","Sans mentir, je l'étais, mais je crois que le ..."
2,"Something like that, yeah.","Quelque chose comme ça, oui."
3,Does my e-mail de dunking on YouTube or on Twi...,C'est mon adresse sur YouTube et sur Twitter.
4,This show was essentially the original X Facto...,C'était essentiellement le précurseur de X Fac...



--- Table for: de-DE ---


,source_text,reference
0,"And he was like, I’ll just take thousands of y...","Und er meinte so: „Ach, das dauerte Tausende v..."
1,Very nice to meet you by the way.,"Ich freue mich übrigens sehr, Sie kennenzulernen."
2,"Of course, you know, it’s not me saying this, ...","Natürlich ist das nicht nur etwas, was ich beh..."
3,It’s a really fascinating thing that seeds do ...,"Es ist wirklich faszinierend, dass sich Samen ..."
4,"It is your founding father, Alex Cooper with.","Hier ist euer Gründervater, Alex Cooper mit"


###### *Automatic **Predictions (Translations)** for all languages (<font color="red">only here: update tuning job IDs manually)*

In [ ]:
# @title Supervised Tuning Batch Translations

import pandas as pd
import vertexai
from vertexai.generative_models import GenerativeModel
from vertexai.tuning import sft

# ADD TUNING JOB IDs MANUUALLY HERE
tuning_job_ids = {
    'es-LA': '3367579076771446784',
    'pt-BR': '3754325694771888128',
    'fr-FR': '8707159384972591104',
    'de-DE': '4430428588830883840'
}

vertexai.init(project=PROJECT_ID, location=LOCATION)
print("✅ SDK Initialized")

# Loop through each language table automatically
print("\n Starting prediction generation for all languages...")
for lang_code, df in ground_truth_dfs.items():
    print(f"\n--- Processing {lang_code} ---")

    try:
        job_id = tuning_job_ids[lang_code]
        job_name = f"projects/{PROJECT_ID}/locations/{LOCATION}/tuningJobs/{job_id}"
        sft_tuning_job = sft.SupervisedTuningJob(job_name)
        tuned_model = GenerativeModel(sft_tuning_job.tuned_model_endpoint_name)
        print(f"✅ Connected to tuned model for {lang_code}.")

    except Exception as e:
        print(f"      - FAILED to load the tuned model for {lang_code}: {e}")
        df['prediction'] = ["ERROR: MODEL LOADING FAILED"] * len(df)
        continue

    # Generate a prediction for each row in the table
    predictions = []
    print(f"   Generating {len(df)} predictions...")
    for source_text in df['source_text']:
        prompt = f"English: {source_text} {lang_code}: "

        try:
            response = tuned_model.generate_content(prompt)
            prediction_text = response.text
            predictions.append(prediction_text.strip())

        except Exception as e:
            print(f"      - Prediction failed for one sample: {e}")
            predictions.append("ERROR: PREDICTION FAILED")

    # Add the list of predictions as a new column to the DataFrame
    df['prediction'] = predictions
    print(f"✅ Predictions added to the '{lang_code}' table.")

# Display top few rows of each language table
for lang_code, df in ground_truth_dfs.items():
    print(f"\n--- Final Table for: {lang_code} ---")
    display(df.head())

✅ SDK Initialized

 Starting prediction generation for all languages...

--- Processing es-LA ---


✅ Connected to tuned model for es-LA.
   Generating 23 predictions...
✅ Predictions added to the 'es-LA' table.

--- Processing pt-BR ---


✅ Connected to tuned model for pt-BR.
   Generating 24 predictions...
✅ Predictions added to the 'pt-BR' table.

--- Processing fr-FR ---


✅ Connected to tuned model for fr-FR.
   Generating 22 predictions...
✅ Predictions added to the 'fr-FR' table.

--- Processing de-DE ---


✅ Connected to tuned model for de-DE.
   Generating 24 predictions...
✅ Predictions added to the 'de-DE' table.

--- Final Table for: es-LA ---


...

,source_text,reference,prediction
0,Is it important?,Isso é importante?,É importante?
1,But I didn't know we had such an impact.,Mas eu não sabia que nosso impacto era tão gra...,Mas eu não sabia que tínhamos tanto impacto.
2,"You know, the fact is that if sleep wasn’t imp...","Sabe, o fato é que, se o sono não fosse import...","Você sabe, o fato é que se o sono não fosse im..."
3,"It is your founding father, Alex Cooper with.",Sou Alex Cooper e você está ouvindo,"É o seu pai fundador, Alex Cooper com."
4,"OK, so I was, I had just taken the LSAT. I wan...","Tá, eu tinha acabado de fazer o LSAT. Eu queri...","OK, então eu estava, eu tinha acabado de fazer..."



--- Final Table for: fr-FR ---


,source_text,reference,prediction
0,But that whole natural selection thing when it...,Tout ce processus de sélection naturelle pour ...,Mais toute cette histoire de sélection naturel...
1,"I'm not kidding you. I was, and I think the en...","Sans mentir, je l'étais, mais je crois que le ...","Je ne plaisante pas. J'étais, et je pense que ..."
2,"Something like that, yeah.","Quelque chose comme ça, oui.","Quelque chose comme ça, oui."
3,Does my e-mail de dunking on YouTube or on Twi...,C'est mon adresse sur YouTube et sur Twitter.,Est-ce que mon e-mail de dunking sur YouTube o...
4,This show was essentially the original X Facto...,C'était essentiellement le précurseur de X Fac...,Ce spectacle était essentiellement le X Factor...



--- Final Table for: de-DE ---


,source_text,reference,prediction
0,"And he was like, I’ll just take thousands of y...","Und er meinte so: „Ach, das dauerte Tausende v...",Und er sagte: „Ich nehme einfach Tausende von ...
1,Very nice to meet you by the way.,"Ich freue mich übrigens sehr, Sie kennenzulernen.","Sehr schön, Sie kennenzulernen."
2,"Of course, you know, it’s not me saying this, ...","Natürlich ist das nicht nur etwas, was ich beh...","Natürlich, das sage ich nicht, sondern das ist..."
3,It’s a really fascinating thing that seeds do ...,"Es ist wirklich faszinierend, dass sich Samen ...","Es ist wirklich faszinierend, dass sich Samen ..."
4,"It is your founding father, Alex Cooper with.","Hier ist euer Gründervater, Alex Cooper mit","Es ist Ihr Gründervater, Alex Cooper mit."
